In [1]:
# importing module
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from myutils import *
from SLBQP import SLBQP
from datastorer import DataStorer
import cvxopt
from cvxopt import matrix
from cvxopt import solvers

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import time
plt.style.use('seaborn')
params = {
    'figure.figsize': (20, 10),
    'legend.fontsize': 'x-large',
    'axes.labelsize': 'x-large',
    'axes.titlesize':'x-large',
    'xtick.labelsize':'x-large',
    'ytick.labelsize':'x-large',
    'axes.titlesize': 'x-large',
}
plt.rcParams.update(params)

# Parametri default

In [3]:
seed = None     # random seed
np.random.seed(seed)
ROSEN = 2
GOLDSTEIN = 1

In [4]:
epsilons = [10**i for i in range(-2, 1)]
upper_bounds = [0.1, 1, 5, 10]
problem_sizes = [200,300,400,600]
n_prob = 10

In [5]:
def generate_multiple_epsilons(feat, targ, size, u, epsilons, n, startdir='./'):
    u_name = str(u).replace(".","")
    for e in epsilons:
        eps_name = str(e).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)

    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)
        for e in epsilons:
            print(f"Solving problem {i} with epsilon {e}")
            eps_name = str(e).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {e=}")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            print("Solving with higher precision")
            sg, xg, vg, itg = SLBQP(K, target, u, e, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN)
            with open(f"{startdir}C{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(vg) + "\n")
            
            

In [6]:
def generate_multiple_upperbounds(feat, targ, size, upper_bounds, epsilon ,n, startdir='./'):
    
    eps_name = str(epsilon).replace(".", "")
    for u in upper_bounds:
        u_name = str(u).replace(".","")
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD", exist_ok=True)
        os.makedirs(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS", exist_ok=True)
        
    for i in range(n):
        print(f"generating problem {i}")
        K, target = sample_transform_problem(feat, targ, size)  
        for u in upper_bounds:
            print(f"Solving problem {i} with U: {u}")
            u_name = str(u).replace(".","")
            ds = DataStorer()
            print(f"\tAbout to run GOLDSTEIN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/GOLD/{i}.csv")
            ds = DataStorer()
            print(f"\tAbout to run ROSEN with {u=}")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-3, maxIter=-1, prj_type=ROSEN, ds=ds)
            ds.save_to_file(f"{startdir}C{u_name}EPS{eps_name}/{size}/ROS/{i}.csv")
            print("Solving with higher precision")
            sg, xg, vg, itg = SLBQP(K, target, u, epsilon, eps=1e-6, maxIter=-1, lmb0=0, d_lmb=2, prj_eps=1e-9, prj_type=GOLDSTEIN)
            with open(f"{startdir}C{u_name}EPS{eps_name}/{size}/optimal_values.txt", "a") as f:
                f.write(str(vg)+"\n")

In [7]:
feat, targ = load_ml_dataset()
t1 = targ[:,0]
t2 = targ[:,1]
targ = t1
ROOT="./RUNS/ML/"

loading from: 
/Users/gerardozinno/repos/ProgettoML/cm/data/ML-CUP19-TR.csv


In [8]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving with higher precision
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving with higher precision
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
Solving with higher precision


In [9]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving with higher precision
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving with higher precision
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving with higher precision
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
Solving with higher precision


In [10]:
feat, targ = load_california_dataset()
targ
ROOT="./RUNS/CAL/"

/usr/local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.california_housing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [11]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving with higher precision
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving with higher precision
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
Solving with higher precision


In [12]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving with higher precision
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving with higher precision
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving with higher precision
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
Solving with higher precision


In [13]:
feat, targ = load_airfoil_dataset()
targ
ROOT="./RUNS/AIRF/"

/Users/gerardozinno/repos/ProgettoML/cm/data/airfoil_self_noise.csv


In [14]:
for size in problem_sizes:
    generate_multiple_epsilons(feat, targ, size, 1, epsilons, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with epsilon 0.01
	About to run GOLDSTEIN with e=0.01
	About to run ROSEN with e=0.01
Solving with higher precision
Solving problem 0 with epsilon 0.1
	About to run GOLDSTEIN with e=0.1
	About to run ROSEN with e=0.1
Solving with higher precision
Solving problem 0 with epsilon 1
	About to run GOLDSTEIN with e=1
	About to run ROSEN with e=1
Solving with higher precision


In [15]:
for size in problem_sizes:
    generate_multiple_upperbounds(feat, targ, size, upper_bounds, 0.1, n_prob, startdir=ROOT)

generating problem 0
Solving problem 0 with U: 0.1
	About to run GOLDSTEIN with u=0.1
	About to run ROSEN with u=0.1
Solving with higher precision
Solving problem 0 with U: 1
	About to run GOLDSTEIN with u=1
	About to run ROSEN with u=1
Solving with higher precision
Solving problem 0 with U: 5
	About to run GOLDSTEIN with u=5
	About to run ROSEN with u=5
Solving with higher precision
Solving problem 0 with U: 10
	About to run GOLDSTEIN with u=10
	About to run ROSEN with u=10
Solving with higher precision
